# Import packages

In [1]:
#!pip install docplex
#!pip install folium

import pandas as pd
import numpy as np
import math 
import folium

# Read data

In [2]:
cases_df = pd.read_csv("cases.csv")
cases_df.head()

,Department,City,Cases,Capacity,Lat,Long
0,Alingsås,Alingsås,3,10,57.929966,12.532968
1,Bollnäs,Bollnäs,1,10,61.318303,16.396794
2,Borås,Borås,4,10,57.721084,12.940741
3,Danderyd,Danderyd,31,50,59.401807,18.061923
4,Eksjö,Eksjö,2,5,57.622698,15.212413


# Visualize data

In [3]:
map_pre = folium.Map(location=[62.212927,15.134684], zoom_start=5)

for index, v in cases_df.iterrows():  
    if v.Capacity > v.Cases + 5:
        color = 'green'
    elif v.Capacity >= v.Cases:
        color='orange'
    elif v.Capacity < v.Cases:
        color='red'

    folium.Circle(location=[v.Lat, v.Long],
                  radius=5000 + v.Cases*500,
                  popup=folium.Popup(html = v.Department + '<br>'+str(v.Cases)+' cases<br>'+ str(v.Capacity) + ' max',
                                     max_width=250,min_width=50),
                  fill=True,color=color).add_to(map_pre)
map_pre

# Process data

In [4]:
cases_df["UnderCapacity"] = cases_df["Cases"] - cases_df["Capacity"]
cases_df["UnderCapacity"] = cases_df["UnderCapacity"].apply(lambda x: max(0,x))
cases_df["SurplusCapacity"] = cases_df["Capacity"] - cases_df["Cases"]
cases_df["SurplusCapacity"] = cases_df["SurplusCapacity"].apply(lambda x: max(0,x))
cases_df = cases_df.set_index(cases_df.Department,drop=False,append=False)
cases_df.head()

,Department,City,Cases,Capacity,Lat,Long,UnderCapacity,SurplusCapacity
Department,,,,,,,,
Alingsås,Alingsås,Alingsås,3,10,57.929966,12.532968,0,7
Bollnäs,Bollnäs,Bollnäs,1,10,61.318303,16.396794,0,9
Borås,Borås,Borås,4,10,57.721084,12.940741,0,6
Danderyd,Danderyd,Danderyd,31,50,59.401807,18.061923,0,19
Eksjö,Eksjö,Eksjö,2,5,57.622698,15.212413,0,3


# Optimization
## Check environment
If CPLEX library isn't present... then you've got some installation stuff to do. I'll probably implement this solution later on in PuLP with an open-source solver. Hang on until then. 

In [5]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

* system is: Darwin 64bit
* Python version 3.7.1, located at: /anaconda3/bin/python
* docplex is present, version is (2, 10, 155)
* CPLEX library is present, version is 12.9.0.0, located at: /Applications/CPLEX_Studio129/cplex/python/3.7/x86-64_osx
* pandas is present, version is 0.25.3


## Set up model, parameters, sets and variables

In [6]:
# Model 
from docplex.mp.model import Model
mdl = Model("PatientAllocations")

# Parameters
NB_PERIODS = 1
MAX_NB_LONG_TRANSFERS_PER_PERIOD = 3
MAX_CASES_PER_LONG_TRANSFERS = 20
MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT = 3
THRESHOLD_FOR_LONG_DISTANCE = 200

# Sets
mdl.deps = list(cases_df.Department)
mdl.transfer_periods = list(range(NB_PERIODS))
mdl.all_periods = list(range(NB_PERIODS+1))

# Variables
mdl.x_vars = {(d1,d2,p): mdl.binary_var(name="x_{0}_{1}_{2}".format(d1,d2,p)) 
              for d1 in mdl.deps for d2 in mdl.deps for p in mdl.transfer_periods}
mdl.y_vars = {(d1,d2,p): mdl.integer_var(name="y_{0}_{1}_{2}".format(d1,d2,p), ub = MAX_CASES_PER_LONG_TRANSFERS) 
              for d1 in mdl.deps for d2 in mdl.deps for p in mdl.transfer_periods}
mdl.o_vars = {(d,p): mdl.integer_var(name="o_{0}_{1}".format(d,p), lb = 0) 
              for d in mdl.deps for p in mdl.all_periods}

## Calculate distances

In [7]:
def distance(lt1, lg1, lt2, lg2):
    R = 6373.0
    lat1 = math.radians(lt1); lon1 = math.radians(lg1);
    lat2 = math.radians(lt2); lon2 = math.radians(lg2);
    dlon = lon2 - lon1; dlat = lat2 - lat1;
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

is_long = {d1:{d2: distance(cases_df.at[d1,"Lat"], 
                            cases_df.at[d1,"Long"], 
                            cases_df.at[d2,"Lat"], 
                            cases_df.at[d2,"Long"]) > THRESHOLD_FOR_LONG_DISTANCE 
               for d2 in mdl.deps} 
           for d1 in mdl.deps}

## Define constraints

In [8]:
# ---------------------------------------------------------------------------------- #
# Short transfers bounds
# ---------------------------------------------------------------------------------- #
for pair in ((d1, d2) for d1 in mdl.deps for d2 in mdl.deps):
    if not is_long[pair[0]][pair[1]]:
        for t in mdl.transfer_periods:
            mdl.add_constraint(ct = mdl.y_vars[pair[0], pair[1], t] <= 1, 
                               ctname = "short_transfer_bound_{0}_{1}_{2}".format(pair[0], pair[1], t))


# ---------------------------------------------------------------------------------- #            
# Set initial state
# ---------------------------------------------------------------------------------- #
for d in mdl.deps: 
    mdl.add_constraint(ct = mdl.o_vars[d, 0] == cases_df.at[d,"Cases"],
                       ctname = "initial_state_{0}".format(d))
    

# ---------------------------------------------------------------------------------- #
# Structural constraint between x_vars and y_vars
# ---------------------------------------------------------------------------------- #
for pair in ((d1, d2) for d1 in mdl.deps for d2 in mdl.deps):
    for t in mdl.transfer_periods:
        mdl.add_constraint(ct = mdl.x_vars[pair[0], pair[1], t] == (mdl.y_vars[pair[0], pair[1], t] >= 1) , 
                           ctname = "use_link_{0}_{1}_{2}".format(pair[0], pair[1], t))

        
# ---------------------------------------------------------------------------------- #
# Number of transfers from a department less than current number of cases
# ---------------------------------------------------------------------------------- #
for d in mdl.deps: 
    for t in mdl.transfer_periods:
        mdl.add_constraint(ct = mdl.sum(mdl.y_vars[d, dx, t] for dx in mdl.deps) <= mdl.o_vars[d,t],
                           ctname = "transfer_less_than_current_cases_{0}_{1}".format(d,t))

    
# ---------------------------------------------------------------------------------- #
# Maximum number of LONG transfers per period
# ---------------------------------------------------------------------------------- #        
for t in mdl.transfer_periods:
    long_transfers = mdl.sum(mdl.x_vars[d1, d2, t] for d1 in mdl.deps for d2 in mdl.deps if is_long[d1][d2])
    mdl.add_constraint(ct = long_transfers <= MAX_NB_LONG_TRANSFERS_PER_PERIOD,
                       ctname = "max_long_transfers_{0}".format(t))
    

# ---------------------------------------------------------------------------------- #
# Maximum number of SHORT transfers per department
# ---------------------------------------------------------------------------------- #  
for d in mdl.deps:
    short_transfers = mdl.sum(mdl.x_vars[dx, d, t] for dx in mdl.deps if not is_long[dx][d] for t in mdl.transfer_periods)
    mdl.add_constraint(ct = short_transfers <= MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT,
                       ctname = "max_short_transfers_{0}".format(d))
    

# ---------------------------------------------------------------------------------- #
# Update number of cases for next period
# ---------------------------------------------------------------------------------- #     
for d in mdl.deps:
    for t in mdl.transfer_periods:
        
        existing_cases = mdl.o_vars[d, t] 
        cases_in = mdl.sum(mdl.y_vars[dx, d, t] for dx in mdl.deps)
        cases_out = mdl.sum(mdl.y_vars[d, dx, t] for dx in mdl.deps)
        organic_growth = 0 #Prediction for organic growth goes here      
        new_cases = existing_cases + cases_in - cases_out + organic_growth
        mdl.add_constraint(ct = mdl.o_vars[d, t+1] == new_cases,
                           ctname = "new_cases_{0}_{1}".format(d,t))

In [9]:
#Inspect constraints
#mdl.get_constraint_by_name("short_transfer_bound_Alingsås_Borås_0")
#mdl.get_constraint_by_name("initial_state_Alingsås")
#mdl.get_constraint_by_name("use_link_Alingsås_Borås_0")
#mdl.get_constraint_by_name("transfer_less_than_current_cases_Alingsås_0")
#mdl.get_constraint_by_name("max_long_transfers_0")
#mdl.get_constraint_by_name("max_short_transfers_Alingsås")
#mdl.get_constraint_by_name("new_cases_Alingsås_0")

## Define objective

In [10]:
final_under_capacity = mdl.sum(mdl.max(0,cases_df.at[d,"Capacity"] - mdl.o_vars[d, NB_PERIODS]) 
                               for d in mdl.deps)
final_over_capacity = mdl.sum(mdl.max(0,mdl.o_vars[d, NB_PERIODS] - cases_df.at[d,"Capacity"]) 
                               for d in mdl.deps)

nb_long_transfers = mdl.sum(mdl.x_vars[d1, d2, t] 
                            for d1 in mdl.deps 
                            for d2 in mdl.deps 
                            if is_long[d1][d2] 
                            for t in mdl.transfer_periods)
nb_short_transfers = mdl.sum(mdl.x_vars[d1, d2, t] 
                             for d1 in mdl.deps 
                             for d2 in mdl.deps 
                             if not is_long[d1][d2] 
                             for t in mdl.transfer_periods)

mdl.minimize(100*final_under_capacity + 10*nb_long_transfers + nb_short_transfers - 10*final_over_capacity)

## Solve model

In [21]:
mdl.set_time_limit(60); #Seconds
mdl.parameters.mip.strategy.probe.set(0);
mdl.parameters.parallel.set(-1); #  opportunistic parallel search mode
mdl.parameters.threads.set(4);

mdl.solve(log_output=True,lex_mipgaps = [0.001])

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
CPXPARAM_Parallel                                -1
CPXPARAM_TimeLimit                               60
CPXPARAM_MIP_Tolerances_MIPGap                   0.001
Reduced MIP has 13777 rows, 12522 columns, and 41550 nonzeros.
Reduced MIP has 7936 binaries, 3968 generals, 0 SOSs, and 4092 indicators.
Presolve time = 0.04 sec. (7.97 ticks)
8 of 8 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 52619.0000.
Tried aggregator 2 times.
MIP Presolve eliminated 2010 rows and 1543 columns.
Aggregator did 5231 substitutions.
Reduced MIP has 4217 rows, 5748 columns, and 21679 nonzeros.
Reduced MIP has 3479 binaries, 2021 generals, 0 SOSs, and 862 indicators.
Presolve time = 0.09 sec. (31.33 ticks)
Probing fixed 0 vars, tightened 248 bounds.
Probing time = 0.09 sec. (6.27 ticks)
Cover probing fixed 0 vars, tightened 67 bounds.
Tried aggregator 2 times.
MIP Presolve

docplex.mp.solution.SolveSolution(obj=52619,values={x_Eskilstuna_K Huddi..

# Interpret solution

## Process solution

In [23]:
edges = [(t, d1, d2, int(mdl.y_vars[d1, d2, t]), is_long[d1][d2]) 
         for t in mdl.transfer_periods 
         for d1 in mdl.deps 
         for d2 in mdl.deps 
         if int(mdl.y_vars[d1, d2, t]) >= 1]
print(edges)

[(0, 'Eskilstuna', 'K Huddinge IVA', 1, False), (0, 'Eskilstuna', 'Karlskoga', 1, False), (0, 'Eskilstuna', 'Karlstad', 1, False), (0, 'Eskilstuna', 'Linköping TIVA', 1, False), (0, 'Eskilstuna', 'Norrköping', 1, False), (0, 'Eskilstuna', 'Norrtälje', 1, False), (0, 'Eskilstuna', 'Nyköping', 1, False), (0, 'Eskilstuna', 'SÖS IVA', 1, False), (0, 'Eskilstuna', 'Uppsala CIVA', 1, False), (0, 'Eskilstuna', 'Örebro TIVA', 1, False), (0, 'Linköping IVA', 'Alingsås', 1, False), (0, 'Linköping IVA', 'Borås', 1, False), (0, 'Linköping IVA', 'Eksjö', 1, False), (0, 'Linköping IVA', 'Kalmar', 1, False), (0, 'Linköping IVA', 'St Göran', 1, False), (0, 'SU Östra', 'Borås', 1, False), (0, 'SU Östra', 'NU Trollhättan', 1, False), (0, 'SU Östra', 'SU CIVA', 1, False), (0, 'Västerås', 'Södertälje', 1, False)]


In [24]:
final = {d: int(mdl.o_vars[d, NB_PERIODS].solution_value) for d in mdl.deps}
cases_df["Final"] = [final[d] for d in mdl.deps]
cases_df["FinalUnderCapacity"] = cases_df["Final"] - cases_df["Capacity"]
cases_df["FinalUnderCapacity"] = cases_df["FinalUnderCapacity"].apply(lambda x: max(0,x))
cases_df["FinalSurplusCapacity"] = cases_df["Capacity"] - cases_df["Final"]
cases_df["FinalSurplusCapacity"] = cases_df["FinalSurplusCapacity"].apply(lambda x: max(0,x))

total_undercapacity_final = cases_df['FinalUnderCapacity'].sum()
total_undercapacity_before = cases_df['UnderCapacity'].sum()
print("Undercapacity before reallocation:",total_undercapacity_before)
print("Undercapacity after reallocation:",total_undercapacity_final)

Undercapacity before reallocation: 19
Undercapacity after reallocation: 0


In [25]:
from folium_scripts import get_arrows, get_bearing

map_fin = folium.Map(location=[62.212927,15.134684], zoom_start=5)

for index, v in cases_df.iterrows():  
    
    # Choose color for circle marker
    if v.Capacity > v.Final + 5:
        color = 'green'
    elif v.Capacity >= v.Final:
        color='orange'
    elif v.Capacity < v.Final:
        color='red'

    # Draw circle marker
    folium.Circle(location=[v.Lat, v.Long],
                  radius=5000 + v.Cases*500,
                  popup=folium.Popup(html = v.Department + '<br>Before: '+str(v.Cases)+'<br>After: '+str(v.Final)+"<br>Capacity: "+str(v.Capacity),max_width=250,min_width=50),
                  fill=True,color=color).add_to(map_fin)
    
    # Draw planned transportation lines
    for (period, d1, d2, nb, il) in edges: 
        coordinates = [[cases_df.at[d1,"Lat"], cases_df.at[d1,"Long"]], 
                       [cases_df.at[d2,"Lat"], cases_df.at[d2,"Long"]]]
        if not il:
            color = 'black'; weight = 2; 
        else:
            color = 'blue'; weight = 2;
        
        pl = folium.PolyLine(coordinates, color=color, weight=weight)
        map_fin.add_child(pl)
        
        arrows = get_arrows(locations=coordinates, color=color, size=4, n_arrows=3)
        for arrow in arrows:
            arrow.add_to(map_fin)

In [26]:
map_fin